## 1_DATA_SELECTION
기업의 기본 제공 데이터에 기업의 외부 데이터를 추가하고 ENVIRON, SOCIAL_DART, GOVERN_DART를 얻는 파이프라인입니다.

## 0. 기본 설정

In [1]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

! pip install {USER_FLAG} --upgrade google-cloud-aiplatform
! pip install {USER_FLAG} --upgrade google-cloud-storage
! pip install dart-fss

In [2]:
# https://oslinux.tistory.com/33
# 크롬 설치 및 웹드라이버 설치. 셀레니움 버전도 낮추기
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | sudo apt-key add - 
!sudo sh -c 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google.list'
!sudo apt-get update 
!sudo apt-get -y install google-chrome-stable

OK
Hit:1 http://packages.cloud.google.com/apt cloud-sdk-buster InRelease
Hit:2 http://deb.debian.org/debian buster InRelease                            
Get:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]          
Get:4 http://security.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:5 http://packages.cloud.google.com/apt google-cloud-packages-archive-keyring-buster InRelease [5553 B]
Get:6 http://packages.cloud.google.com/apt gcsfuse-buster InRelease [5391 B]   
Hit:7 https://dl.google.com/linux/chrome/deb stable InRelease                  
Hit:8 https://packages.cloud.google.com/apt google-fast-socket InRelease       
Hit:9 http://packages.cloud.google.com/apt google-compute-engine-buster-stable InRelease
Get:11 http://deb.debian.org/debian buster-backports InRelease [51.4 kB]       
Hit:12 https://download.docker.com/linux/debian buster InRelease               
Hit:10 https://packages.cloud.google.com/apt kubernetes-xenial InRelease
Get:13 https:/

In [3]:
!google-chrome --version ## 해당 버전에 맞는 chromewebdriver를 다운받고 webdriver 폴더 안에 넣어주세요.

Google Chrome 106.0.5249.119 


In [4]:
# 설치된 크롬 버전에 맞는 드라이버 깔기. 버전 상시적으로 수정해야 합니다.
!wget https://chromedriver.storage.googleapis.com/105.0.5195.52/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
# zip 파일 지우고 파일 옮기기
!rm -f chromedriver_linux64.zip
!mv chromedriver webdriver/chromedriver

--2022-10-15 16:38:54--  https://chromedriver.storage.googleapis.com/105.0.5195.52/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 74.125.197.128, 2607:f8b0:400e:c03::80
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7315600 (7.0M) [application/zip]
Saving to: ‘chromedriver_linux64.zip’

chromedriver_linux6 100%[===================>]   6.98M  --.-KB/s    in 0.1s    

2022-10-15 16:38:54 (51.7 MB/s) - ‘chromedriver_linux64.zip’ saved [7315600/7315600]

Archive:  chromedriver_linux64.zip
  inflating: chromedriver            


In [5]:
!pip uninstall selenium --y
!pip install selenium==3.10
!pip3 install lxml

Found existing installation: selenium 3.10.0
Uninstalling selenium-3.10.0:
  Successfully uninstalled selenium-3.10.0
  Using cached selenium-3.10.0-py2.py3-none-any.whl (943 kB)


In [6]:
# 설치가 끝나면 커널을 재시작해야합니다.
import os

if not os.getenv("IS_TESTING"):
# Automatically restart kernel after installs
    import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
# 세팅을 다시 로드
import os
import pandas as pd

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
import warnings
import os
import joblib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from google.cloud import aiplatform, storage

from google.cloud import storage
from google.cloud import bigquery

import pandas as pd
from sklearn import datasets

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
    
# 지역, GCS 클라이언트, BQ 클라이언트 설정
REGION = 'asia-northeast3'
gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

# bucket 이름 지정: "프로젝트 이름-bucket"
BUCKET_NAME=PROJECT_ID + "-bucket"
if not gcs.lookup_bucket(BUCKET_NAME):
    bucketDef = gcs.bucket(BUCKET_NAME)
    bucket = gcs.create_bucket(bucketDef, project=PROJECT_ID, location=REGION)
    print(bucket)
else:
    print(gcs.lookup_bucket(BUCKET_NAME))

Project ID:  bigfesta2022-esg-team6-proj
<Bucket: bigfesta2022-esg-team6-proj-bucket>


In [2]:
from selenium import webdriver
import sys
options = webdriver.ChromeOptions()
options.add_argument('--headless')    
options.add_argument('--window-size=1920x1080')# Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('webdriver/chromedriver', options=options)

In [3]:
# import 라이브러리
import os
from google.cloud import aiplatform
from google.cloud import bigquery
import pandas as pd
import numpy as np
import dart_fss as dart
from dart_fss import get_corp_list
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from selenium.webdriver.support.select import Select
import time
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import sys
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import warnings
import numpy as np
import dart_fss as dart
from dart_fss import get_corp_list
from dart_fss.auth import get_api_key
from dart_fss.utils import request

## 1. code

In [4]:
# bigquery에 있는 데이터를 데이터프레임으로 불러옵니다.
def get_dataframe(bq_table): 
    # bq_table에는 "스키마 이름.불러오고싶은 테이블 이름"이 들어가면 됩니다.
    
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    project_number = shell_output[0]
    bqclient = bigquery.Client(project=project_number)
    
    table = bigquery.TableReference.from_string(
        project_number+"."+bq_table
    )
    
    rows = bqclient.list_rows(
        table
    )
    
    dataframe = rows.to_dataframe(
        create_bqstorage_client=True,
    )

    return dataframe

In [5]:
#데이터프레임을 빅쿼리에 저장합니다.
def to_bigquery(
    dataframe,
    table_name: str,
    bucket_name: str
):
    # dataframe은 big query에 저장하고 싶은 데이터 프레임을 넣어줍니다.
    # table_name에는 "스키마 이름.생성하고 싶은 테이블 이름"이 들어가면 됩니다.
    # 스키마는 미리 빅쿼리에 생성해두셔야 합니다.
    # bucket name은 0.기본설정에서 선언해준 bucket name을 넣으면 됩니다. 
    
    client = bigquery.Client()
    column_list=dataframe.columns
    query_column_list=""
    
    #blob = storage.blob.Blob.from_string(f"""gs://{bucket_name}/{table_name.split(".")[-1]}.csv""", client=storage.Client())
    dataframe.to_csv(f"""gs://{bucket_name}/{table_name.split(".")[-1]}.csv""",index=False,encoding="utf-8")
    #blob.upload_from_file(temp)
    
    for name in column_list:
        query_column_list=query_column_list+name+" STRING,"
        
    query = f"""
    LOAD DATA OVERWRITE {table_name}(
    {query_column_list}
        )
    FROM FILES (
      skip_leading_rows=1,
      allow_jagged_rows=true,
      format = 'CSV',
      encoding='utf-8',
      field_delimiter=',',
      uris = ['gs://{bucket_name}/{table_name.split(".")[-1]}.csv']);"""
    
    query_job = client.query(query)
    results = query_job.result()

In [6]:
# 기업의 추가적인 데이터(업종 등)를 DART로부터 불러오는 코드입니다. 
def get_company_info(
    total_corp: str,
    industry_code: str,
    corp_asset_sum: str,
    dart_api
):  
    import numpy as np
    import dart_fss as dart
    from dart_fss import get_corp_list
    from dart_fss.auth import get_api_key
    from dart_fss.utils import request
    
    corp_df = get_dataframe(total_corp)
    industry_code=get_dataframe(industry_code)
    corp_asset_sum=get_dataframe(corp_asset_sum)

    key=dart_api
    dart.set_api_key(api_key=key)
    crp_list = get_corp_list()
    # 기업 데이터프레임 기준으로 기업 정보 저장. 해당 정보를 담은 데이터프레임 생성
    total_corp_final=pd.DataFrame(columns=["code","name","market","kor_name","eng_name","stock_corp_code","stock_name","industry_code"])
    for i, stock_code in enumerate(corp_df["code"]):
        #print("{0}번째 {1} 기업입니다.".format(i,stock_code))
        stock_code=stock_code.replace("'","")
        try: # 기업 정보가 있다면
            corp_code=crp_list.find_by_stock_code(stock_code).corp_code
            # DART 에 공시된 회사 리스트 불러오기
            result=dart.api.filings.get_corp_info(corp_code)
            total_corp_final.loc[i]=[corp_df["code"][i],corp_df["name"][i],corp_df["market"][i], result["corp_name"],result["corp_name_eng"],str("'"+result["corp_code"]+"'"),result["stock_name"],str("'"+result["induty_code"]+"'")]
        except: # 기업 정보가 없다면
            total_corp_final.loc[i]=[corp_df["code"][i],corp_df["name"][i],corp_df["market"][i],np.NaN,np.NaN,np.NaN,np.NaN,np.NaN]
    
    # industry_code 데이터프레임과  join 후 industry 정보 붙이기
    total_corp_final["industry_med_code"]="'"+total_corp_final["industry_code"].str[1:3]+"'"
    total_corp_final["industry_sub_code"]="'"+total_corp_final["industry_code"].str[1:4]+"'"
    total_corp_final = pd.merge(total_corp_final,industry_code, how='left', left_on='industry_med_code', right_on='indust_code').rename(columns={"indust_name":"industry_med_name"})
    total_corp_final= pd.merge(total_corp_final,industry_code, how='left', left_on='industry_sub_code', right_on='indust_code').rename(columns={"indust_name":"industry_sub_name"})
    
    # corp_asset_sum 데이터프레임과 join 후 corp_asset_sum 정보 붙이기
    total_corp_final=pd.merge(total_corp_final,corp_asset_sum[["code","asset_sum"]],how="left",on="code")
    #print(total_corp_final)
    
    return total_corp_final

In [7]:
# DART로부터 불러온 지배구조보고서 접수번호를 이용하여 지배구조보고서 다운로드 링크를 추출하는 함수입니다.
def govern_report_crawl(corp_df,webdriver_path):
    driver = webdriver.Chrome(webdriver_path, options=options)
    govern_report=pd.DataFrame(columns=["code","stock_corp_code","name","report_src","rcept_no_G","date_G"])
    for i, corp_row in corp_df.iterrows():
        if(corp_row["rcept_no_G"] is not None):
            url=str("https://dart.fss.or.kr/dsaf001/main.do?rcpNo="+str(int(corp_row["rcept_no_G"])))
            html = urlopen(url)
            bsObject = BeautifulSoup(html, "lxml", from_encoding='utf-8')
            value=bsObject.select_one("#att > option:nth-child(2)")["value"]
            
            url_2=str("https://dart.fss.or.kr/dsaf001/main.do?"+str(value))
            #print(url_2)         
            driver.get(url_2)
            time.sleep(1)
    
            url_3 = driver.find_element_by_css_selector('#ifrm').get_attribute("src")
            #print(url_3)
            driver.get(url_3)

            report_url=None
            
            report_urls=driver.find_elements_by_xpath('//a')
            for url_temp in report_urls:
                url_href=url_temp.get_attribute("href")
                url_text=url_temp.text
                if(("지배구조" in url_text)&("보고서" in url_text)):
                    report_url=url_href
                    break
            
            if report_url==None:
                report_url=driver.find_element_by_xpath('//a').get_attribute("href")
                
            #print(report_url)

            govern_report.loc[i]=[corp_row["code"],corp_row["stock_corp_code"],corp_row["name"],report_url,corp_row["rcept_no_G"],corp_row["date_G"]]
            
        else:
            govern_report.loc[i]=[corp_row["code"],corp_row["stock_corp_code"],corp_row["name"],np.NaN,corp_row["rcept_no_G"],corp_row["date_G"]]
            
            
    return govern_report
# DART로부터 불러온 지속가능경영보고서 접수번호를 이용하여 지속가능경영보고서 다운로드 링크를 추출하는 함수입니다.
def social_report_crawl(corp_df,webdriver_path):
    driver = webdriver.Chrome(webdriver_path, options=options)
    social_report=pd.DataFrame(columns=["code","stock_corp_code","name","report_src","rcept_no_S","date_S"])
    for i, corp_row in corp_df.iterrows():
        #print(i)
        if(corp_row["rcept_no_S"] is not None):
            url=str("https://dart.fss.or.kr/dsaf001/main.do?rcpNo="+str(int(corp_row["rcept_no_S"])))
            html = urlopen(url)
            bsObject = BeautifulSoup(html, "lxml", from_encoding='utf-8')
            value=bsObject.select_one("#att > option:nth-child(2)")["value"]
            
            url_2=str("https://dart.fss.or.kr/dsaf001/main.do?"+str(value))
            #print(url_2)         
            driver.get(url_2)
            time.sleep(1)
    
            url_3 = driver.find_element_by_css_selector('#ifrm').get_attribute("src")
            #print(url_3)
        
            driver.get(url_3)
            report_url=np.NaN
            report_url=driver.find_element_by_xpath('//a').get_attribute("href")
        
            #print(report_url)
            social_report.loc[i]=[corp_row["code"],corp_row["stock_corp_code"],corp_row["name"],report_url,corp_row["rcept_no_S"],corp_row["date_S"]]
            
        else:
            social_report.loc[i]=[corp_row["code"],corp_row["stock_corp_code"],corp_row["name"],np.NaN,corp_row["rcept_no_S"],corp_row["date_S"]]
    return social_report

In [8]:
# DART 로부터 보고서 접수번호를 얻는 함수입니다.
def get_rcept_num(
    corp,
    year,
    dart_api,
):
    import numpy as np
    import pandas as pd
    import dart_fss as dart
    from dart_fss import get_corp_list
    
    key=dart_api
    dart.set_api_key(api_key=key) ## api 인증키를 설정합니다.

    crp_list = get_corp_list() ## 기업 리스트를 dart에서 받아옵니다.
    
    ## 지배구조 보고서 , 지속가능경영보고서 번호와 날짜를 저장할 데이터프레임을 생성합니다.
    rcept_no_data = pd.DataFrame(columns=["code","stock_corp_code","name","rcept_no_G","date_G","rcept_no_S","date_S"])
    #print("지배구조 보고서를 탐색합니다.")
    for i, stock_code in enumerate(corp["stock_corp_code"]):
        rceptG = 123456
        dateG = 123456
        rceptS = 123456
        dateS = 123456
        ## 보고서 탐색 과정에서 보고서를 이미 찾은 경우 다음 단계로 넘어가기 위해서는 논리 연산이 필요합니다.
        ## 그래서 임의의 숫자를 대입함으로써 논리 연산이 가능하게 했습니다.
        
        ## 지배구조 보고서와 지속가능경영보고서 모두 해당 기간의 보고서가 100개 이상이면 100개만 탐색하는 경우가 있어
        ## 100개 이상의 보고서가 없게끔 기간별로 나누어서 보고서를 탐색하였습니다.

        if(stock_code is not None):
            corp_code = stock_code.replace("'","")
        else :
            corp_code = np.NaN
        try : ## 종목 코드가 있다면
            re = dart.api.filings.search_filings(corp_code = corp_code , bgn_de = str(year)+'0101', end_de = str(year)+'0901', page_count=100)
            ## 받은 year의 1월 1일부터 9월 1일까지의 보고서를 가져옴
            if(re['total_count'] < 100): ## 해당 기간의 보고서가 100개 이하인 경우
                for j in range(len(re['list'])):
                    if('지배구조보고서' in re['list'][j]['report_nm']):
                        rceptG = re['list'][j]['rcept_no']
                        dateG = re['list'][j]['rcept_dt']

                if((rceptG != 123456) & (dateG != 123456)): ## 보고서를 찾은 경우
                    rcept_no_data.loc[i] = [corp['code'][i],corp['stock_corp_code'][i],corp['name'][i],rceptG,dateG,1,1] ## 아직 탐색대상이 아닌 사회부분 보고서 번호와 날짜에 대해서는 1을 넣어줍니다.
                    

            else : ## year년 보고서가 100개 넘는 경우
                re = dart.api.filings.search_filings(corp_code = corp_code , bgn_de = str(year)+'0501', end_de = str(year)+'0901', page_count=100) ## 5월부터로

                for k in range(len(re['list'])):
                    if('지배구조보고서' in re['list'][k]['report_nm']):
                        rceptG = re['list'][k]['rcept_no']
                        dateG = re['list'][k]['rcept_dt']


                if((rceptG != 123456) & (dateG != 123456)): ## 찾은 경우
                    rcept_no_data.loc[i] = [corp['code'][i],corp['stock_corp_code'][i],corp['name'][i],rceptG,dateG,1,1] 
                     

                else : ## 5월부터 해봤는데 못 찾은 경우
                    re = dart.api.filings.search_filings(corp_code = corp_code , bgn_de = str(year)+'0101', end_de = str(year)+'0430', page_count=100)

                    for d in range(len(re['list'])):
                        if('지배구조보고서' in re['list'][d]['report_nm']):
                            rceptG = re['list'][d]['rcept_no']
                            dateG = re['list'][d]['rcept_dt']

                    if((rceptG != 123456) & (dateG != 123456)): ## 찾은 경우
                        rcept_no_data.loc[i] = [corp['code'][i],corp['stock_corp_code'][i],corp['name'][i],rceptG,dateG,1,1] 
                        

            ## 여기까지가 year년에서 찾기는 경우

            if(rceptG == 123456): ## year년에서 못찾은 경우 -> year 기준으로 작년 보고서를 탐색하고 100개 이상의 보고서 탐색을 방지하고자 3개월 단위로 끊어서 탐색합니다
                re = dart.api.filings.search_filings(corp_code = corp_code , bgn_de = str(year - 1) + '0901', end_de = str(year - 1)+'1231', page_count=100)

                for l in range(len(re['list'])):
                    if('지배구조보고서' in re['list'][l]['report_nm']):
                        rceptG = re['list'][l]['rcept_no']
                        dateG = re['list'][l]['rcept_dt']


                if((rceptG != 123456) & (dateG != 123456)): ## 찾은 경우
                    rcept_no_data.loc[i] = [corp['code'][i],corp['stock_corp_code'][i],corp['name'][i],rceptG,dateG,1,1] 
                     


            if(rceptG == 123456): ## ( year - 1 )년 9월 이후에서 못찾은 경우
                re = dart.api.filings.search_filings(corp_code = corp_code , bgn_de = str(year - 1) + '0601', end_de = str(year - 1) + '0831', page_count=100)

                for a in range(len(re['list'])):
                    if('지배구조보고서' in re['list'][a]['report_nm']):
                        rceptG = re['list'][a]['rcept_no']
                        dateG = re['list'][a]['rcept_dt']


                if((rceptG != 123456) & (dateG != 123456)): ## 찾은 경우
                    rcept_no_data.loc[i] = [corp['code'][i],corp['stock_corp_code'][i],corp['name'][i],rceptG,dateG,1,1] 
                


            if(rceptG == 123456): ## ( year - 1 )년 6월까지에서 못찾은 경우
                re = dart.api.filings.search_filings(corp_code = corp_code , bgn_de = str(year - 1) + '0301', end_de = str(year - 1) + '0531', page_count=100)

                for b in range(len(re['list'])):
                    if('지배구조보고서' in re['list'][b]['report_nm']):
                        rceptG = re['list'][b]['rcept_no']
                        dateG = re['list'][b]['rcept_dt']


                if((rceptG != 123456) & (dateG != 123456)): ## 찾은 경우
                    rcept_no_data.loc[i] = [corp['code'][i],corp['stock_corp_code'][i],corp['name'][i],rceptG,dateG,1,1] 
                    


            if(rceptG == 123456): ## (year - 1)년 3월까지 에서 못찾은 경우
                re = dart.api.filings.search_filings(corp_code = corp_code , bgn_de = str(year - 1) + '0101', end_de = str(year - 1) + '0228', page_count=100)

                for c in range(len(re['list'])):
                    if('지배구조보고서' in re['list'][c]['report_nm']):
                        rceptG = re['list'][c]['rcept_no']
                        dateG = re['list'][c]['rcept_dt']


                if((rceptG != 123456) & (dateG != 123456)): ## 찾은 경우
                    rcept_no_data.loc[i] = [corp['code'][i],corp['stock_corp_code'][i],corp['name'][i],rceptG,dateG,1,1] 
                    

            if(rceptG == 123456): ## 마지막까지 못찾은 경우
                rcept_no_data.loc[i] = [corp['code'][i],corp['stock_corp_code'][i],corp['name'][i],np.NaN,np.NaN,1,1] ## 보고서를 최종적으로 탐색하지 못한 경우에는 np.NaN 을 넣어줍니다
                

        except : ## 이외의 오류가 발생한 경우 결측치를 넣어줍니다.
            rcept_no_data.loc[i] = [corp['code'][i],corp['stock_corp_code'][i],corp['name'][i],np.NaN,np.NaN,np.NaN,np.NaN]

    
    
    ## 지속가능경영보고서 또한 앞선 지배구조보고서 탐색 과정과 유사한 방식으로 진행합니다.

    for i_2, stock_code_2 in enumerate(corp["stock_corp_code"]):
    
        rceptG = 123456
        dateG = 123456
        rceptS = 123456
        dateS = 123456
        #print('{0}번째 기업입니다'.format(i_2))
    
        if(stock_code_2 is not None):
            corp_code = stock_code_2.replace("'","")
        else :
            corp_code = np.NaN
        try : ## 종목 코드가 존재한다면
            re = dart.api.filings.search_filings(corp_code = corp_code , bgn_de = str(year)+'0101', end_de = str(year)+'0901', page_count=100)
    
            if(re['total_count'] < 100): ## year년 보고서가 100개 이하인 경우
                for j_2 in range(len(re['list'])):
                    if('지속가능경영보고서' in re['list'][j_2]['report_nm']):
                        rceptS = re['list'][j_2]['rcept_no']
                        dateS = re['list'][j_2]['rcept_dt']
                    
                if((rceptS != 123456) & (dateS != 123456)): ## 찾은 경우
                    rcept_no_data.loc[i_2] = [corp['code'][i_2],corp['stock_corp_code'][i_2],corp['name'][i_2],rcept_no_data['rcept_no_G'][i_2],rcept_no_data['date_G'][i_2],rceptS,dateS] 
                    
            else : ## year년 보고서가 100개 넘는 경우
                re = dart.api.filings.search_filings(corp_code = corp_code , bgn_de = str(year)+'0501', end_de = str(year)+'0901', page_count=100) ## 5월부터로
        
                for k_2 in range(len(re['list'])):
                    if('지속가능경영보고서' in re['list'][k_2]['report_nm']):
                        rceptS = re['list'][k_2]['rcept_no']
                        dateS = re['list'][k_2]['rcept_dt']
                
        
                if((rceptS != 123456) & (dateS != 123456)): ## 찾은 경우
                    rcept_no_data.loc[i_2] = [corp['code'][i_2],corp['stock_corp_code'][i_2],corp['name'][i_2],rcept_no_data['rcept_no_G'][i_2],rcept_no_data['date_G'][i_2],rceptS,dateS]
                    
            
                else : ## 5월부터 해봤는데 못 찾은 경우
                    re = dart.api.filings.search_filings(corp_code = corp_code , bgn_de = str(year)+'0101', end_de = str(year)+'0430', page_count=100)
                
                    for d_2 in range(len(re['list'])):
                        if('지속가능경영보고서' in re['list'][d_2]['report_nm']):
                            rceptS = re['list'][d_2]['rcept_no']
                            dateS = re['list'][d_2]['rcept_dt']
                
                    if((rceptS != 123456) & (dateS != 123456)): ## 찾은 경우
                        rcept_no_data.loc[i_2] = [corp['code'][i_2],corp['stock_corp_code'][i_2],corp['name'][i_2],rcept_no_data['rcept_no_G'][i_2],rcept_no_data['date_G'][i_2],rceptS,dateS] 
                        
            
        ## 여기까지가 year년에서 찾기는 경우
    
            if(rceptS == 123456): ## year년에서 못찾은 경우
                re = dart.api.filings.search_filings(corp_code = corp_code , bgn_de = str(year-1)+'0901', end_de = str(year-1)+'1231', page_count=100)
        
                for l_2 in range(len(re['list'])):
                    if('지속가능경영보고서' in re['list'][l_2]['report_nm']):
                        rceptS = re['list'][l_2]['rcept_no']
                        dateS = re['list'][l_2]['rcept_dt']
                
        
                if((rceptS != 123456) & (dateS != 123456)): ## 찾은 경우
                    rcept_no_data.loc[i_2] = [corp['code'][i_2],corp['stock_corp_code'][i_2],corp['name'][i_2],rcept_no_data['rcept_no_G'][i_2],rcept_no_data['date_G'][i_2],rceptS,dateS] 
                    
                
            
            if(rceptS == 123456): ## (year-1)년 9월 이후에서 못찾은 경우
                re = dart.api.filings.search_filings(corp_code = corp_code , bgn_de = str(year-1)+'0601', end_de = str(year-1)+'0831', page_count=100)
        
                for a_2 in range(len(re['list'])):
                    if('지속가능경영보고서' in re['list'][a_2]['report_nm']):
                        rceptS = re['list'][a_2]['rcept_no']
                        dateS = re['list'][a_2]['rcept_dt']
                
        
                if((rceptS != 123456) & (dateS != 123456)): ## 찾은 경우
                    rcept_no_data.loc[i_2] = [corp['code'][i_2],corp['stock_corp_code'][i_2],corp['name'][i_2],rcept_no_data['rcept_no_G'][i_2],rcept_no_data['date_G'][i_2],rceptS,dateS] 
                    
               
            
            if(rceptS == 123456): ## (year - 1)년 6월까지에서 못찾은 경우
                re = dart.api.filings.search_filings(corp_code = corp_code , bgn_de = str(year-1)+'0301', end_de = str(year-1)+'0531', page_count=100)
        
                for b_2 in range(len(re['list'])):
                    if('지속가능경영보고서' in re['list'][b_2]['report_nm']):
                        rceptS = re['list'][b_2]['rcept_no']
                        dateS = re['list'][b_2]['rcept_dt']
                
        
                if((rceptS != 123456) & (dateS != 123456)): ## 찾은 경우
                    rcept_no_data.loc[i_2] = [corp['code'][i_2],corp['stock_corp_code'][i_2],corp['name'][i_2],rcept_no_data['rcept_no_G'][i_2],rcept_no_data['date_G'][i_2],rceptS,dateS] 
                    
                
            
            if(rceptS == 123456): ## (year - 1)년 3월까지 에서 못찾은 경우
                re = dart.api.filings.search_filings(corp_code = corp_code , bgn_de = str(year-1)+'0101', end_de = str(year-1)+'0228', page_count=100)
        
                for c_2 in range(len(re['list'])):
                    if('지속가능경영보고서' in re['list'][c_2]['report_nm']):
                        rceptS = re['list'][c_2]['rcept_no']
                        dateS = re['list'][c_2]['rcept_dt']
                
        
                if((rceptS != 123456) & (dateS != 123456)): ## 찾은 경우
                    rcept_no_data.loc[i_2] = [corp['code'][i_2],corp['stock_corp_code'][i_2],corp['name'][i_2],rcept_no_data['rcept_no_G'][i_2],rcept_no_data['date_G'][i_2],rceptS,dateS]
                    
               
            if(rceptS == 123456): ## 마지막까지 못찾은 경우
                rcept_no_data.loc[i_2] = [corp['code'][i_2],corp['stock_corp_code'][i_2],corp['name'][i_2],rcept_no_data['rcept_no_G'][i_2],rcept_no_data['date_G'][i_2],np.NaN,np.NaN]
                

        except :
            rcept_no_data.loc[i] = [corp['code'][i],corp['stock_corp_code'][i],corp['name'][i],rcept_no_data['rcept_no_G'][i],rcept_no_data['date_G'][i],np.NaN,np.NaN]
            #print("오류발생")
            
    return(rcept_no_data)

In [9]:
# 환경정보시스템에서 환경정보를 크롤링해오는 코드입니다.
def environ_crawl(corp_df, year,webdriver_path):
    driver = webdriver.Chrome(webdriver_path, options=options)
    url = "https://www.env-info.kr/member/open/companyTotalInfoSearch.do" 
    driver.get(url)
    # 전체 기업 정보 저장할 데이터프레임
    corp_df_env=pd.DataFrame(columns=["code","year","raw","water","energy","air_pollu","water_pollu","waste","chemical","water_reuse","waste_reuse"])
    for i, corp_row in corp_df.iterrows():
        # 기업정보 일시적으로 저장할 데이터프레임
        #######################################################
        temp_df_env=pd.DataFrame(columns=["raw","water","energy","air_pollu","water_pollu","waste","chemical","water_reuse","waste_reuse"])
        #print("{0}번째 {1} 기업입니다.".format(str(i),str(corp_row["code"])))
        # 년도 선택
        select=Select(driver.find_element_by_id("schYear"))
        select.select_by_value(str(year)) #select index value
        # 200개씩#pageRows
        select=Select(driver.find_element_by_id("pageRows"))
        select.select_by_value(str(200)) #select index value
        # 회사명 검색
        element = driver.find_element_by_id('schCompNm')
        element.send_keys(str(corp_row["name"]))
        #print(str(corp_row["name"]))
        element.send_keys("\n")
        time.sleep(1)
        if(len(driver.find_elements_by_css_selector("#tblDocList > tbody > tr > td:nth-child(7) > a"))>0):# 첫 번째 이름으로 검사
            button_list=driver.find_elements_by_css_selector("#tblDocList > tbody > tr > td:nth-child(7) > a")
            tabs = driver.window_handles
            #print(tabs)
            # 기업 사업장 정보를 임시로 저장할 데이터프레임 형성
            for button in button_list:
                # 기업의 각 사업장 선택하고, 등장하는 팝업창의 자료 크롤링
                #time.sleep(1)
                button.send_keys(Keys.ENTER)
                tabs = driver.window_handles
                driver.switch_to_window(tabs[1])
                # url 하나씩 접속하여 
                # 크롤링해야하는 변수 : 속성값에 해당 값 포함하면 크롤링, 없으면 np.Nan 저장!
                # ⑨ 원부자재 사용량(단위: ton) 
                try:
                    raw_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "원부자재(누적질량 70% 이상) 사용 실적")]')
                    raw_result=raw_data[0].find_element_by_css_selector("table > tbody > tr:nth-child(2) > td").get_attribute("innerText")
                    raw_result=raw_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                    raw_data=float(raw_result)
                except:
                    raw_result=np.NaN
                # ④ 용수 사용량(단위: ton) 
                try:
                    water_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "용수 사용·재활용량 실적")]')
                    water_result=water_data[0].find_element_by_css_selector("table > tbody > tr:nth-child(1) > td").get_attribute("innerText")
                    water_result=water_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                    water_result=float(water_result)
                except:
                    water_result=np.NaN
                # ⑤ 에너지 사용량(단위: toe) #inquiry08 > table > tbody > tr > td
                try:
                    energy_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "에너지 총 사용량")]')
                    energy_result=energy_data[0].find_element_by_css_selector("table > tbody > tr > td").get_attribute("innerText")
                    energy_result=energy_result.strip().replace("\n","").replace("\t","").split(" ")
                    result_index=energy_result.index("TJ")-1
                    energy_result=energy_result[result_index].replace(",","")
                    energy_result=float(energy_result)
                except:
                    energy_result=np.NaN
                # ② 대기오염물질 배출량(단위: ton) 
                try: 
                    air_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "대기오염물질총배출량")]')
                    air_result=air_data[0].find_element_by_css_selector("table > tbody > tr:nth-child(4) > td").get_attribute("innerText")
                    air_result=air_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                    air_result=float(air_result)
                except:
                    air_result=np.NaN
                # ① 수질오염물질 배출량(단위: ton)
                try:
                    w_pollu_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "수질오염물질총배출량")]')
                    w_pollu_result=w_pollu_data[0].find_element_by_css_selector("table > tbody > tr:nth-child(6) > td").get_attribute("innerText")
                    w_pollu_result=w_pollu_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                    w_pollu_result=float(w_pollu_result)
                except:
                    w_pollu_result=np.NaN
                # ③ 폐기물 발생량(단위: ton)
                try:
                    waste_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "폐기물 배출·재활용 실적")]')
                    waste_result=waste_data[0].find_element_by_css_selector("table > tbody > tr:nth-child(5) > td").get_attribute("innerText")
                    waste_result=waste_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                    waste_result=float(waste_result)
                except:
                    waste_result=np.NaN
                # ⑥ 화학물질 배출량(단위: ton) 
                try:
                    chemical_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "화학물질 배출실적")]')
                    chemical_result=chemical_data[0].find_element_by_css_selector("table > tbody > tr > td").get_attribute("innerText")
                    chemical_result=chemical_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                    chemical_result=float(chemical_result)
                except:
                    chemical_result=np.NaN
                # ⑦ 용수 재활용량(단위: %) 
                try:
                    w_reuse_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "용수 사용·재활용량 실적")]')
                    w_reuse_result=w_reuse_data[0].find_element_by_css_selector("table > tbody > tr:nth-child(2) > td").get_attribute("innerText")
                    w_reuse_result=w_reuse_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                    w_reuse_result=float(w_reuse_result)
                except:
                    w_reuse_result=np.NaN
                # ⑧ 폐기물재활용량(단위: %) 
                try:
                    wa_reuse_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "폐기물 배출·재활용 실적")]')
                    wa_reuse_result=wa_reuse_data[0].find_element_by_css_selector("table > tbody > tr:nth-child(4) > td").get_attribute("innerText")
                    wa_reuse_result=wa_reuse_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                    wa_reuse_result=float(wa_reuse_result)
                except:
                    wa_reuse_result=np.NaN
                # ⑩ 기타 
                # law_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "원부자재 사용량")]')
                # law_result=law_data[0].find_element_by_css_selector("table > tbody > tr:nth-child(2) > td").get_attribute("innerText")
                # law_result=law_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                #print("raw_result,water_result,energy_result,air_result,w_pollu_result,waste_result,chemical_result,w_reuse_result,wa_reuse_result")
                #print(raw_result,water_result,energy_result,air_result,w_pollu_result,waste_result,chemical_result,w_reuse_result,wa_reuse_result)
                ##############################################
                #df.append(pd.Series(row, index=df.columns), ignore_index=True)
                temp_df_env=temp_df_env.append(pd.Series([float(raw_result),float(water_result),float(energy_result),float(air_result),float(w_pollu_result),float(waste_result),float(chemical_result),float(w_reuse_result),float(wa_reuse_result)],index=temp_df_env.columns),ignore_index=True)
                #print(temp_df_env)
                driver.close()
                tabs = driver.window_handles
                driver.switch_to_window(tabs[0])
            # 각 사업장별 합 최종 데이터 프레임에 저장
            #print(temp_df_env)
            corp_df_env=corp_df_env.append(pd.Series([corp_row["code"],year]+list(temp_df_env.sum(axis=0)),index=corp_df_env.columns),ignore_index=True)
            #print(corp_df_env)
            ######################################################
        else:# 검색결과가 없다면 두번째 이름으로 재검사
            if(corp_row["kor_name"]!=None):
                # 년도 선택
                select=Select(driver.find_element_by_id("schYear"))
                select.select_by_value(str(year)) #select index value
                # 200개씩#pageRows
                select=Select(driver.find_element_by_id("pageRows"))
                select.select_by_value(str(200)) #select index value
                # 회사명 검색
                element = driver.find_element_by_id('schCompNm')
                element.clear()
                element.send_keys(str(corp_row["kor_name"].replace("(주)","")))
                #print(str(corp_row["kor_name"].replace("(주)","")))
                element.send_keys("\n")
                time.sleep(1)
                #print(len(driver.find_elements_by_css_selector("#tblDocList > tbody > tr > td:nth-child(7) > a")))
                if(len(driver.find_elements_by_css_selector("#tblDocList > tbody > tr > td:nth-child(7) > a"))>0):# 첫 번째 이름으로 검사
                    button_list=driver.find_elements_by_css_selector("#tblDocList > tbody > tr > td:nth-child(7) > a")
                    tabs = driver.window_handles
                    #print(tabs)
                    # 기업 사업장 정보를 임시로 저장할 데이터프레임 형성
                    for button in button_list:
                        # 기업의 각 사업장 선택하고, 등장하는 팝업창의 자료 크롤링
                        #time.sleep(1)
                        button.send_keys(Keys.ENTER)
                        tabs = driver.window_handles
                        driver.switch_to_window(tabs[1])
                        # url 하나씩 접속하여 
                        # 크롤링해야하는 변수 : 속성값에 해당 값 포함하면 크롤링, 없으면 np.Nan 저장!
                        # ⑨ 원부자재 사용량(단위: ton) 
                        try:
                            raw_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "원부자재(누적질량 70% 이상) 사용 실적")]')
                            raw_result=raw_data[0].find_element_by_css_selector("table > tbody > tr:nth-child(2) > td").get_attribute("innerText")
                            raw_result=raw_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                            raw_result=float(raw_result)
                        except:
                            raw_result=np.NaN
                        # ④ 용수 사용량(단위: ton) 
                        try:
                            water_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "용수 사용·재활용량 실적")]')
                            water_result=water_data[0].find_element_by_css_selector("table > tbody > tr:nth-child(1) > td").get_attribute("innerText")
                            water_result=water_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                            water_result=float(water_result)
                        except:
                            water_result=np.NaN
                        # ⑤ 에너지 사용량(단위: toe) #inquiry08 > table > tbody > tr > td
                        try:
                            energy_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "에너지 총 사용량")]')
                            energy_result=energy_data[0].find_element_by_css_selector("table > tbody > tr > td").get_attribute("innerText")
                            energy_result=energy_result.strip().replace("\n","").replace("\t","").split(" ")
                            result_index=energy_result.index("TJ")-1
                            energy_result=energy_result[result_index].replace(",","")
                            energy_result=float(energy_result)
                        except:
                            energy_result=np.NaN
                        # ② 대기오염물질 배출량(단위: ton) 
                        try: 
                            air_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "대기오염물질총배출량")]')
                            air_result=air_data[0].find_element_by_css_selector("table > tbody > tr:nth-child(4) > td").get_attribute("innerText")
                            air_result=air_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                            air_result=float(air_result)
                        except:
                            air_result=np.NaN
                        # ① 수질오염물질 배출량(단위: ton)
                        try:
                            w_pollu_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "수질오염물질총배출량")]')
                            w_pollu_result=w_pollu_data[0].find_element_by_css_selector("table > tbody > tr:nth-child(6) > td").get_attribute("innerText")
                            w_pollu_result=w_pollu_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                            w_pollu_result=float(w_pollu_result)
                        except:
                            w_pollu_result=np.NaN
                        # ③ 폐기물 발생량(단위: ton)
                        try:
                            waste_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "폐기물 배출·재활용 실적")]')
                            waste_result=waste_data[0].find_element_by_css_selector("table > tbody > tr:nth-child(5) > td").get_attribute("innerText")
                            waste_result=waste_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                            waste_result=float(waste_result)
                        except:
                            waste_result=np.NaN
                        # ⑥ 화학물질 배출량(단위: ton) 
                        try:
                            chemical_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "화학물질 배출실적")]')
                            chemical_result=chemical_data[0].find_element_by_css_selector("table > tbody > tr > td").get_attribute("innerText")
                            chemical_result=chemical_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                            chemical_result=float(chemical_result)
                        except:
                            chemical_result=np.NaN
                        # ⑦ 용수 재활용량(단위: %) 
                        try:
                            w_reuse_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "용수 사용·재활용량 실적")]')
                            w_reuse_result=w_reuse_data[0].find_element_by_css_selector("table > tbody > tr:nth-child(2) > td").get_attribute("innerText")
                            w_reuse_result=w_reuse_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                            w_reuse_result=float(w_reuse_result)
                        except:
                            w_reuse_result=np.NaN
                        # ⑧ 폐기물재활용량(단위: %) 
                        try:
                            wa_reuse_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "폐기물 배출·재활용 실적")]')
                            wa_reuse_result=wa_reuse_data[0].find_element_by_css_selector("table > tbody > tr:nth-child(4) > td").get_attribute("innerText")
                            wa_reuse_result=wa_reuse_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                            wa_reuse_result=float(wa_reuse_result)
                        except:
                            wa_reuse_result=np.NaN
                        # ⑩ 기타 
                        # law_data=driver.find_elements_by_xpath('/html/body/div/form[2]/div/div/div[2]/div[contains(., "원부자재 사용량")]')
                        # law_result=law_data[0].find_element_by_css_selector("table > tbody > tr:nth-child(2) > td").get_attribute("innerText")
                        # law_result=law_result.strip().replace("\n","").replace("\t","").split(" ")[0].replace(",","")
                        #print("raw_result,water_result,energy_result,air_result,w_pollu_result,waste_result,chemical_result,w_reuse_result,wa_reuse_result")
                        #print(raw_result,water_result,energy_result,air_result,w_pollu_result,waste_result,chemical_result,w_reuse_result,wa_reuse_result)

                        driver.close()
                        tabs = driver.window_handles
                        driver.switch_to_window(tabs[0])
                        temp_df_env=temp_df_env.append(pd.Series([float(raw_result),float(water_result),float(energy_result),float(air_result),float(w_pollu_result),float(waste_result),float(chemical_result),float(w_reuse_result),float(wa_reuse_result)],index=temp_df_env.columns),ignore_index=True)
                        #print(temp_df_env)
                        ##################################################
                    # 각 사업장별 평균값 최종 데이터 프레임에 저장
                    ########################################################
                    #print(temp_df_env)
                    corp_df_env=corp_df_env.append(pd.Series([corp_row["code"],year]+list(temp_df_env.sum(axis=0)),index=corp_df_env.columns),ignore_index=True)

                else: # 그것마저 없다면 none 저장
                    raw_result=water_result=energy_result=air_result=w_pollu_result=waste_result=chemical_result=w_reuse_result=wa_reuse_result=np.NaN
                    temp_df_env=temp_df_env.append(pd.Series([float(raw_result),float(water_result),float(energy_result),float(air_result),float(w_pollu_result),float(waste_result),float(chemical_result),float(w_reuse_result),float(wa_reuse_result)],index=temp_df_env.columns),ignore_index=True)
                    corp_df_env=corp_df_env.append(pd.Series([corp_row["code"],year]+list(temp_df_env.sum(axis=0)),index=corp_df_env.columns),ignore_index=True)
                    ############################################################
            else:
                raw_result=water_result=energy_result=air_result=w_pollu_result=waste_result=chemical_result=w_reuse_result=wa_reuse_result=np.NaN
                temp_df_env=temp_df_env.append(pd.Series([float(raw_result),float(water_result),float(energy_result),float(air_result),float(w_pollu_result),float(waste_result),float(chemical_result),float(w_reuse_result),float(wa_reuse_result)],index=temp_df_env.columns),ignore_index=True)
                corp_df_env=corp_df_env.append(pd.Series([corp_row["code"],year]+list(temp_df_env.sum(axis=0)),index=corp_df_env.columns),ignore_index=True)
                    ############################################################
    return corp_df_env

In [10]:
# DART에서 제공되는 파생변수를 스크랩핑하는 코드입니다.
def social_dart_crawl(corp_df, year,dart_api):    
    import dart_fss as dart_fss
    from dart_fss import get_corp_list
    key=dart_api

    dart_fss.set_api_key(api_key=key) ## api 인증키를 설정합니다.

    crp_list = get_corp_list() ## 기업 리스트를 dart에서 받아옵니다.
    
    import pandas as pd
    import numpy as np
    
    df1 = pd.DataFrame(columns = ["code" , "M_reg" , "F_reg" , "M_totalmoney" , "F_totalmoney" , "M_meanmoney" , "F_meanmoney" , "M_sm" , "F_sm"])
    ## df1 이라는 사회부분 파생변수를 생성하는데 필요한 데이터들을 담을 데이터프레임을 생성합니다.
    for i, stock_code in enumerate(corp_df["code"]):
        try : ## 해당 code가 NaN이 아니라면
            stock_code_new = stock_code.replace("'","")
            try : ## crp_list에 해당 stock_code를 가진 기업이 있다면
                male = 2 
                corp_code = crp_list.find_by_stock_code(stock_code_new).corp_code
                result = dart_fss.api.info.emp_sttus(corp_code, bsns_year = str(year), reprt_code = '11011', api_key = key)
                result = pd.DataFrame(result['list'])
                if((result['sexdstn'][0] == '남')|(result['sexdstn'][0] == '남자')|(result['sexdstn'][0] == '남성')|(result['sexdstn'][0] == 'M')|(result['sexdstn'][0] == 'Male')) :
                    male = 0 ## 남자 row가 위에 있는 경우
                else :
                    male = 1 ## 여자 row가 위에 있는 경우
                df1.loc[i] = [corp_df["code"][i],result['rgllbr_co'][male],result['rgllbr_co'][1 - male],result['fyer_salary_totamt'][male],result['fyer_salary_totamt'][1-male],result['jan_salary_am'][male],result['jan_salary_am'][1-male],result['sm'][male],result['sm'][1-male]]
                ## print("파생변수 채워넣기 성공")
            except :
                df1.loc[i] = [corp_df['code'][i],np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN]
        except :
            df1.loc[i] = [corp_df['code'][i],np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN]  
    
    # 1,234 같은 천단위 구분 쉼표 제거
    df1['M_reg']= df1['M_reg'].str.replace(',', '')
    df1['F_reg']= df1['F_reg'].str.replace(',', '')
    df1['M_totalmoney']= df1['M_totalmoney'].str.replace(',', '')
    df1['F_totalmoney']= df1['F_totalmoney'].str.replace(',', '')
    df1['M_meanmoney']= df1['M_meanmoney'].str.replace(',', '')
    df1['F_meanmoney']= df1['F_meanmoney'].str.replace(',', '')
    df1['M_sm']= df1['M_sm'].str.replace(',', '')
    df1['F_sm']= df1['F_sm'].str.replace(',', '')
    
    social_Dart = pd.DataFrame(columns = ["code" , "fullt_prob" , "female_prob" , "fullt_female_prob" , "female_wage_prob"])
    
    ## 밑 for 문 과정을 통해, dart에서 제공하는 최초의 파생변수들을 이용하여 저희의 모델링에 필요한 사회부분 파생변수들을 생성하게 됩니다.
    for k in range(len(df1['code'])): 
        try :
            reg_prob = (float(df1['M_reg'][k]) + float(df1['F_reg'][k])) / (float(df1['M_sm'][k]) + float(df1['F_sm'][k])) ## 정규직 비율 : (남성 정규직 + 여성 정규직) / (남성 총 직원 + 여성 총 직원)
            female_prob = float(df1['F_sm'][k]) / (float(df1['M_sm'][k]) + float(df1['F_sm'][k])) ## 여성 직원 비율 : 여성 총 직원 / (남성 총 직원 + 여성 총 직원)
            reg_female_prob = float(df1['F_reg'][k]) / (float(df1['M_reg'][k]) + float(df1['F_reg'][k])) ## 정규직 여성 비율 : 여성 정규직 수 / (남성 정규직 수 + 여성 정규직 수)
            female_money_prob = float(df1['F_meanmoney'][k]) / float(df1['M_meanmoney'][k]) ## 남성 대비 여성 임금 : 여성 평균 임금 / 남성 평균 임금
            social_Dart.loc[k] = [df1["code"][k], reg_prob , female_prob , reg_female_prob , female_money_prob] 
        except :
            social_Dart.loc[k] = [df1["code"][k] , np.NaN, np.NaN, np.NaN, np.NaN]
            
    return(social_Dart)

In [11]:
def govern_dart_crawl(corp_df, year,dart_api):
    import dart_fss as dart
    from dart_fss import get_corp_list
    import pandas as pd
    import numpy as np
    key=dart_api
    dart.set_api_key(api_key=key) ## api 인증키를 설정합니다.
    crp_list = get_corp_list() ## 기업 리스트를 dart에서 받아옵니다.
    
    # 1)  이사회 사외이사 비율
    # 조직과 독립적인 사외이사 비율을 측정하여 평가하며 사외이사의 비율이 높을수록 높은 점수를 받는 경향이 있습니다.

    df1 = pd.DataFrame(columns=["code","total","saway"]) ## 기업이름과 총 이사 수, 사외이사 수를 저장할 데이터프레임을 만듭니다.

    for i, stock_code in enumerate(corp_df["code"]):
        try : ## 해당 code가 결측값이 아니라면
            stock_code_new = stock_code.replace("'","") ## 앞자리에 0이 사라지는 걸 방지하기 위해 붙혀준 따옴표를 제거합니다.
            try : ## crp_list에 해당 stock_code를 가진 기업이 있다면
                corp_code = crp_list.find_by_stock_code(stock_code_new).corp_code
                result = dart.api.info.outcmpny_drctr_nd_change_sttus(corp_code, bsns_year = str(year) , reprt_code = '11011', api_key = key)
                df1.loc[i] = [corp_df["code"][i],result["list"][0]["drctr_co"],result["list"][0]["otcmp_drctr_co"]]
            except : ## 없다면 결측값을 채워넣습니다
                df1.loc[i] = [corp_df["code"][i],np.NaN,np.NaN]
        except :
            df1.loc[i] = [np.NaN,np.NaN,np.NaN]

    ## 확인결과, 사외이사가 없는 경우 '-' 로 표시됨 -> '-' 로 표기된 경우 0으로 변환
    for j in range(len(df1['saway'])):
        if(df1['saway'][j] == '-'):
            df1['saway'][j] = 0
            
    df1_final = pd.DataFrame(columns=["code","Outer_percentage"])
    for k in range(len(df1['total'])):
        df1_final.loc[k] = [df1['code'][k],  ((float(df1['saway'][k]) / float(df1['total'][k])))] ## (사외이사 / 전체 이사) -> 사외이사 비율
    
    
    # 2)  이사회 성별 다양성
    # 이사회 구성원 대비 전체 여성의 비율을 측정하여 여성의 비율이 일정 수준 이상일수록 높은 점수를 받는 경향이 있습니다.

    df2= pd.DataFrame(columns=["code","male", "female"]) ## 기업이름과 이사 중 남성, 여성의 수를 저장할 데이터프레임을 형성합니다.

    for l, stock_code in enumerate(corp_df["code"]):
        try : ## 해당 code가 none이 아니라면
            stock_code_new = stock_code.replace("'","") ## 앞자리에 0이 사라지는 걸 방지하기 위해 붙혀준 따옴표를 제거합니다.
            #print("{0}번째 {1} 기업입니다.".format(l,stock_code_new))
            try : ## crp_list에 해당 stock_code를 가진 기업이 있다면
                corp_code = crp_list.find_by_stock_code(stock_code_new).corp_code
                result2 = dart.api.info.exctv_sttus(corp_code, bsns_year = str(year) , reprt_code = '11011' , api_key = key)
                male = 0
                female = 0
                for m in range(len(result2['list'])):
                    if((result2['list'][m]['rgist_exctv_at'] == '사외이사') | (result2['list'][m]['rgist_exctv_at'] == '사내이사')) :
                        if(result2['list'][m]['sexdstn'] == '남'): ## 남성 임원 한명 추가
                            male = male + 1
                        elif(result2['list'][m]['sexdstn'] == '여'): ## 여성 임원 한명 추가
                            female = female + 1
                df2.loc[l] = [corp_df['code'][l],male,female]
            except : ## 없다면 결측값을 채워넣습니다
                df2.loc[l] = [corp_df['code'][l],np.NaN,np.NaN]
        except :
            df2.loc[l] = [np.NaN,np.NaN,np.NaN]

    df2_final = pd.DataFrame(columns=["code","female_percentage"])
    ## 밑 for문에서 남성과 여성 임원의 합이 0인 경우를 고려하여 적절하게 전처리하여, 여성 임원 비율을 얻어냅니다.
    for a in range(len(df2['code'])):
        if(df2['code'][a] == np.NaN):
            df2_final.loc[a] = [np.NaN , np.NaN]
        elif((df2['female'][a] + df2['male'][a])!=0):
            df2_final.loc[a] = [df2['code'][a], ( df2['female'][a] / (df2['female'][a] + df2['male'][a]))]
        elif((df2['female'][a] + df2['male'][a])==0):
            df2_final.loc[a] = [df2['code'][a], 0]

            
    data_G_Dart = pd.merge(df1_final, df2_final) ## 사외이사 비율을 담은 데이터와 여성 임원 비율을 담은 데이터를 merge함으로써 최종 데이터셋을 획득합니다.
    
    return(data_G_Dart)


In [12]:
# 실행하기 전에 프로젝트 이름의 bigquery에 common과 esg_data라는 스키마를 만들고
# 미리 준비해둔 1) 기업명과 기업코드가 담긴 total_corp, 
# 2) 산업코드와 산업명이 담긴 industry_v10, 3) 기업별 자산총계가 담긴 asset_sum이라는 테이블을 만들어야 합니다.
# 크롤링할 때 쓸 webdriver_path를 지정해줘야 합니다. 
# 마지막으로 DART openAPI 홈페이지에 접속하셔서 본인 API_KEY를 넣어주시면 됩니다.

# pipeline은 위 모든 함수를 한번에 실행하는 코드입니다.
def pipeline(
    project: str = PROJECT_ID,
    region: str = REGION,
    bucket: str=BUCKET_NAME,
    year:int=2021,
    total_corp: str="common.total_corp",
    industry_code: str="common.industry_v10",
    corp_asset_sum: str="common.corp_asset_sum",
    webdriver_path:str="webdriver/chromedriver",
    dart_api: str="c76b1f22ef317617c15f5c3efa535c9540a374eb"
):

    # 1.get_company_info
    print("get_company_info 진행 중")
    start_time=time.time()
    print(dart_api)
    total_corp_final=get_company_info(total_corp,industry_code,corp_asset_sum,dart_api)
    end_time=time.time()
    dur_time=end_time-start_time
    print("duration: "+str(dur_time))
    
    # 2.to_bigquery(get_company_info)
    to_bigquery(total_corp_final,"common.total_corp_final",bucket)
    
    # 3.report_num
    total_corp_final = get_dataframe("common.total_corp_final")
    print("get_recept_num 진행 중")
    start_time=time.time()
    report_num=get_rcept_num(total_corp_final,year,dart_api)
    end_time=time.time()
    dur_time=end_time-start_time
    print("duration: "+str(dur_time))
    
    # 4.to_bigquery(report_num)
    to_bigquery(report_num,"common.report_num",bucket)
    
    # 5.social_report_crawl
    report_num= get_dataframe("common.report_num")
    print("social_report_crawl 진행 중")
    start_time=time.time()
    social_report_src=social_report_crawl(report_num,webdriver_path)
    end_time=time.time()
    dur_time=end_time-start_time
    print("duration: "+str(dur_time))
    to_bigquery(social_report_src,"esg_data.social_report_src",bucket)
    
    # 6. govern_report_crawl
    print("govern_report_crawl 진행 중")
    start_time=time.time()
    govern_report_src=govern_report_crawl(report_num,webdriver_path)
    end_time=time.time()
    dur_time=end_time-start_time
    print("duration: "+str(dur_time))
    to_bigquery(govern_report_src,"esg_data.govern_report_src",bucket)
    
    # 7. environ_crawl
    print("3개년 environ_crawl 진행 중 (대략 8시간 소요) ")
    total_corp_final = get_dataframe("common.total_corp_final")
    start_time=time.time()
    environ_1=environ_crawl(total_corp_final,year-1,webdriver_path)
    environ_2=environ_crawl(total_corp_final,year-2,webdriver_path)
    environ_3=environ_crawl(total_corp_final,year-3,webdriver_path)
    end_time=time.time()
    dur_time=end_time-start_time
    print("duration: "+str(dur_time))

    # to_bigquery(report_num)
    to_bigquery(environ_1,"esg_data.environ_"+str(year-1),bucket)
    to_bigquery(environ_2,"esg_data.environ_"+str(year-2),bucket)
    to_bigquery(environ_3,"esg_data.environ_"+str(year-3),bucket)

    # 8. social_dart_crawl
    print("social_dart_crawl 진행 중")
    total_corp_final = get_dataframe("common.total_corp_final")
    start_time=time.time()
    social_dart=social_dart_crawl(total_corp_final,year,dart_api)
    end_time=time.time()
    dur_time=end_time-start_time
    print("duration: "+str(dur_time))
    # to_bigquery(report_num)
    to_bigquery(social_dart,"esg_data.social_dart",bucket)

    # 9. govern_dart_crawl
    print("govern_dart_crawl 진행 중")
    start_time=time.time()
    govern_dart=govern_dart_crawl(total_corp_final,year,dart_api)
    end_time=time.time()
    dur_time=end_time-start_time
    print("duration: "+str(dur_time))
    # to_bigquery(report_num)
    to_bigquery(govern_dart,"esg_data.govern_dart",bucket)


In [14]:
# parameter 종류
# project: str = PROJECT_ID,
# region: str = REGION,
# bucket: str=BUCKET_NAME,
# year:int=2021,
# total_corp: str="common.total_corp",
# industry_code: str="common.industry_v10",
# corp_asset_sum: str="common.corp_asset_sum",
# webdriver_path:str="webdriver/chromedriver",
# dart_api: str="c76b1f22ef317617c15f5c3efa535c9540a374eb"

# 실행하기 전에 프로젝트 이름의 bigquery에 common과 esg_data라는 스키마를 만들고
# 미리 준비해둔 1) 기업명과 기업코드가 담긴 total_corp, 
# 2) 산업코드와 산업명이 담긴 industry_v10, 3) 기업별 자산총계가 담긴 asset_sum이라는 테이블을 만들어야 합니다.
# 크롤링할 때 쓸 webdriver_path를 지정해줘야 합니다. 
# 마지막으로 DART openAPI 홈페이지에 접속하셔서 본인 API_KEY를 넣어주시면 됩니다.

# pipeline은 위 모든 함수를 한번에 실행하는 코드입니다.
pipeline(PROJECT_ID,REGION,BUCKET_NAME,2021,"common.total_corp","common.corp_asset_sum","common.corp_asset_sum","webdriver/chromedriver","c76b1f22ef317617c15f5c3efa535c9540a374eb")